In [ ]:
# Author: Janssen Kotah
# snRNAseq analysis for WT/SHIP1 KO mice as part of Matera et al. project
# Code adapted from Seurat analysis template by Mirjam Koster

In [1]:
library(Seurat)
library(dplyr)
library(patchwork)
library(ggplot2)


Attaching SeuratObject


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
sample_list_seu <- readRDS("002B_listed_samples_after_SoupX_unprocessed_with_metadata.rds")
sample_list_seu

$JK1
An object of class Seurat 
21963 features across 8087 samples within 1 assay 
Active assay: RNA (21963 features, 0 variable features)

$JK2
An object of class Seurat 
22333 features across 8227 samples within 1 assay 
Active assay: RNA (22333 features, 0 variable features)

$JK3
An object of class Seurat 
22289 features across 10399 samples within 1 assay 
Active assay: RNA (22289 features, 0 variable features)

$JK4
An object of class Seurat 
21371 features across 5840 samples within 1 assay 
Active assay: RNA (21371 features, 0 variable features)

$JK5
An object of class Seurat 
22171 features across 7129 samples within 1 assay 
Active assay: RNA (22171 features, 0 variable features)

$JK6
An object of class Seurat 
21902 features across 5817 samples within 1 assay 
Active assay: RNA (21902 features, 0 variable features)

$JK7
An object of class Seurat 
19433 features across 884 samples within 1 assay 
Active assay: RNA (19433 features, 0 variable features)

$JK8
An object of cl

In [3]:
Output.dir = "./002C_QC/"

In [4]:
sample_list <- names(sample_list_seu)
sample_list

[1] "JK1" "JK2" "JK3" "JK4" "JK5" "JK6" "JK7" "JK8"

In [5]:
## QUALITY CONTROL
pdf(file = paste0(Output.dir, "1.1 Counts-Features and histogram per sample.pdf"))
for (each in sample_list) {
    to_use <- sample_list_seu[[each]] 
    plot(x = to_use$nCount_RNA, y = to_use$nFeature_RNA)
    title(each)
    
    hist(as.matrix(log10(to_use@assays$RNA@counts)), main = each)
}

dev.off()


Warning message in .sparse2dense(x):
“sparse->dense coercion: allocating vector of size 1.3 GiB”
Warning message in .sparse2dense(x):
“sparse->dense coercion: allocating vector of size 1.4 GiB”
Warning message in .sparse2dense(x):
“sparse->dense coercion: allocating vector of size 1.7 GiB”
Warning message in .sparse2dense(x):
“sparse->dense coercion: allocating vector of size 1.2 GiB”


png 
  2

In [6]:
## PRE-PROCESSING
# Percentage mitochondrial & ribosomal genes (metadata)
# Ribosomal pattern for mice: "^Rp[sl]"

for (each in sample_list) {
    sample_list_seu[[each]][["percent.mito"]] <- PercentageFeatureSet(sample_list_seu[[each]], pattern = "^mt-")
    sample_list_seu[[each]][["percent.ribo"]] <- PercentageFeatureSet(sample_list_seu[[each]], pattern = "^Rp[sl]")
}

# Visualize QC metrics as a violin plot
pdf(file = paste0(Output.dir,"1.2 Quality control plots per sample.pdf"), 
    title = "Quality control plots")
for (each in sample_list) {
  print(VlnPlot(sample_list_seu[[each]], features = c("nCount_RNA", "nFeature_RNA", "percent.mito", "percent.ribo"), 
                ncol = 2, pt.size = 0,
       group.by = "sample"))
}

dev.off()

png 
  2

In [7]:
for (each in sample_list) {
  sample_list_seu[[each]]@meta.data %>% nrow() %>% print()
}

[1] 8087
[1] 8227
[1] 10399
[1] 5840
[1] 7129
[1] 5817
[1] 884
[1] 4936


In [8]:
for (each in sample_list) {
  subset(sample_list_seu[[each]], subset = percent.mito < 5) %>% .@meta.data %>% nrow() %>% print()
}

[1] 8087
[1] 8227
[1] 10398
[1] 5840
[1] 7129
[1] 5816
[1] 882
[1] 4936


In [9]:
#20240209 based on Thomas' code
# Subset cells with <5% mito genes
# Filter for > 200 features and < 6000 features
# Filter for nCount between 200 and 20000

#for (each in sample_list) {
#  sample_list_seu[[each]] <- subset(sample_list_seu[[each]], subset = percent.mito < 5 & nFeature_RNA > 200 & 
#                              nFeature_RNA < 6000 & nCount_RNA < 20000 & nCount_RNA > 200) 
#}

#20240301 filter only for mito, but also doesn't change that much
for (each in sample_list) {
  sample_list_seu[[each]] <- subset(sample_list_seu[[each]], subset = percent.mito < 5) 
}

In [10]:
##NORMALISATION
# default: normalization.method = "LogNormalize", scale.factor = 10000
pdf(file = paste0(Output.dir, "1.3 Normalised data per sample.pdf"))
for (each in sample_list) {
  sample_list_seu[[each]] <- NormalizeData(sample_list_seu[[each]])
  
  hist(as.matrix(log2(sample_list_seu[[each]][["RNA"]]@data)), main = paste0("Normalised data: sample ", each))
}
dev.off()

Warning message in .sparse2dense(x):
“sparse->dense coercion: allocating vector of size 1.3 GiB”
Warning message in .sparse2dense(x):
“sparse->dense coercion: allocating vector of size 1.4 GiB”
Warning message in .sparse2dense(x):
“sparse->dense coercion: allocating vector of size 1.7 GiB”
Warning message in .sparse2dense(x):
“sparse->dense coercion: allocating vector of size 1.2 GiB”


png 
  2

In [11]:
##FEATURE SELECTION
# Identification of highly variable features
# default: selection.method = "vst", nfeature = 2000

pdf(file = paste0(Output.dir, "1.4 Variable features plot per sample.pdf"))
for (each in sample_list) {
  sample_list_seu[[each]] <- FindVariableFeatures(sample_list_seu[[each]])
  
  # plot variable features with top 10 labeled
  top10 <- head(VariableFeatures(sample_list_seu[[each]]), 10)
  plot1 <- VariableFeaturePlot(sample_list_seu[[each]])
  
  print(LabelPoints(plot = plot1, points = top10, repel = TRUE, xnudge = 0, ynudge = 0))
}
dev.off()

png 
  2

In [12]:
##SCALE FEATURES
# Scale data & run PCA separately per sample
dim = 1:30 #Check if 30 dimensions is enough by inspecting the Elbow plots

pdf(file = paste0(Output.dir, "1.5 Scaled data + plots per sample.pdf"))
for (each in sample_list) {
  sample_list_seu[[each]] <- ScaleData(sample_list_seu[[each]])
  sample_list_seu[[each]] <- RunPCA(sample_list_seu[[each]])
  
  print(DimPlot(object = sample_list_seu[[each]], reduction = "pca"))
  print(ElbowPlot(sample_list_seu[[each]] , ndims = 50))
  
  sample_list_seu[[each]]  <- FindNeighbors(sample_list_seu[[each]] , dims = dim)
  sample_list_seu[[each]] <- FindClusters(object = sample_list_seu[[each]] , resolution = 0.1)
  sample_list_seu[[each]] <- RunUMAP(sample_list_seu[[each]] , dims = dim, raster = T)
  
  print(DimPlot(object = sample_list_seu[[each]] , reduction = "umap"))
}
dev.off()


Centering and scaling data matrix

PC_ 1 
Positive:  Ralyl, Celf4, Hs6st3, Adcy2, Nrxn3, Cacna2d3, Dpp10, Fat3, Galnt18, Ldb2 
	   Ryr3, Nrg1, Cntn5, Kcnq5, Tenm3, Ntm, Mef2c, Cdh8, Gabrg3, Chrm3 
	   Frmpd4, Kctd16, Gria1, St6galnac5, Adcy8, Cntn3, Nwd2, Cntnap2, Hdac9, Grm8 
Negative:  Plp1, Tcf7l2, Itpr2, Bcas1, St18, Qk, Tns3, Gab1, Kank1, Gjc3 
	   Wipf1, Plekhg1, Sox2ot, Mob3b, Abtb2, 9630013A20Rik, Zfp536, 1700047M11Rik, Chd7, Erbb4 
	   Tcf7l1, Gramd3, Mag, Enpp6, Fa2h, Sox10, Mbp, Prkcq, Cnksr3, Ugt8a 
PC_ 2 
Positive:  Bnc2, Cped1, Gulp1, Eya2, Tbx15, Prdm6, Nxn, Fbxl7, Zic4, Phldb2 
	   Bmp6, Dock5, Slc47a1, Uaca, Neat1, Sned1, Anxa3, Pdgfd, Egfr, 2610307P16Rik 
	   Foxd1, Cgnl1, Notch2, 9530026P05Rik, Piezo2, Ranbp3l, Adamtsl3, Col1a2, Slc38a2, Itgbl1 
Negative:  Plp1, St18, Mbp, Gjc3, Mag, Bcas1, 1700047M11Rik, Nfasc, Mobp, 9630013A20Rik 
	   Elovl7, Ugt8a, Fa2h, Cldn14, Enpp6, Tns3, Prima1, Gm16168, Kif19a, Plxnb3 
	   Tspan2, Cnksr3, Sox10, Gm10863, Sirt2, Bfsp2, Sox2ot,

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 8087
Number of edges: 301926

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9865
Number of communities: 18
Elapsed time: 0 seconds


Warning message:
“The following arguments are not used: raster”
Warning message:
“The default method for RunUMAP has changed from calling Python UMAP via reticulate to the R-native UWOT using the cosine metric
To use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'
This message will be shown once per session”
23:53:22 UMAP embedding parameters a = 0.9922 b = 1.112

23:53:22 Read 8087 rows and found 30 numeric columns

23:53:22 Using Annoy for neighbor search, n_neighbors = 30

23:53:22 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

23:53:23 Writing NN index file to temp file /tmp/Rtmp1N4Nhb/file2186675f2e90bc

23:53:23 Searching Annoy index using 1 thread, search_k = 3000

23:53:25 Annoy recall = 100%

23:53:25 Commencing smooth 

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 8227
Number of edges: 298944

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9818
Number of communities: 17
Elapsed time: 0 seconds


Warning message:
“The following arguments are not used: raster”
23:53:46 UMAP embedding parameters a = 0.9922 b = 1.112

23:53:46 Read 8227 rows and found 30 numeric columns

23:53:46 Using Annoy for neighbor search, n_neighbors = 30

23:53:46 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

23:53:47 Writing NN index file to temp file /tmp/Rtmp1N4Nhb/file2186673068d93c

23:53:47 Searching Annoy index using 1 thread, search_k = 3000

23:53:49 Annoy recall = 100%

23:53:49 Commencing smooth kNN distance calibration using 1 thread
 with target n_neighbors = 30

23:53:50 Initializing from normalized Laplacian + noise (using irlba)

23:53:51 Commencing optimization for 500 epochs, with 350294 positive edges

23:54:01 Optimization finished

Centering and scaling data matrix



...

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 882
Number of edges: 22635

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9688
Number of communities: 10
Elapsed time: 0 seconds


Warning message:
“The following arguments are not used: raster”
23:55:21 UMAP embedding parameters a = 0.9922 b = 1.112

23:55:21 Read 882 rows and found 30 numeric columns

23:55:21 Using Annoy for neighbor search, n_neighbors = 30

23:55:21 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

23:55:21 Writing NN index file to temp file /tmp/Rtmp1N4Nhb/file218667252b2d7d

23:55:21 Searching Annoy index using 1 thread, search_k = 3000

23:55:21 Annoy recall = 100%

23:55:21 Commencing smooth kNN distance calibration using 1 thread
 with target n_neighbors = 30

23:55:22 Initializing from normalized Laplacian + noise (using irlba)

23:55:22 Commencing optimization for 500 epochs, with 31438 positive edges

23:55:23 Optimization finished

Centering and scaling data matrix

PC

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 4936
Number of edges: 165968

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.9776
Number of communities: 10
Elapsed time: 0 seconds


Warning message:
“The following arguments are not used: raster”
23:55:27 UMAP embedding parameters a = 0.9922 b = 1.112

23:55:27 Read 4936 rows and found 30 numeric columns

23:55:27 Using Annoy for neighbor search, n_neighbors = 30

23:55:27 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

23:55:28 Writing NN index file to temp file /tmp/Rtmp1N4Nhb/file2186674b2bde85

23:55:28 Searching Annoy index using 1 thread, search_k = 3000

23:55:29 Annoy recall = 100%

23:55:29 Commencing smooth kNN distance calibration using 1 thread
 with target n_neighbors = 30

23:55:30 Initializing from normalized Laplacian + noise (using irlba)

23:55:31 Commencing optimization for 500 epochs, with 200692 positive edges

23:55:37 Optimization finished



png 
  2

In [20]:
pdf(file = paste0(Output.dir, "1.6 Scrublet results per sample.pdf"))
for (each in sample_list) {
    #p1 = DimPlot(sample_list_seu[[each]], label = FALSE, raster = T)
    p2 = DimPlot(sample_list_seu[[each]], group.by = "predicted_doublets", raster = T) 

    print(p2 + plot_annotation(title = paste0('Predicted doublets from scrublet, sample ', each)))
}

dev.off()


png 
  2

In [19]:
saveRDS(sample_list_seu, "002C_listed_samples_after_SoupX_QCprocessed.rds") 
